In [134]:
import copy
import numpy as np
from matplotlib import pyplot as plt

from PTI.ReadDataFiles import PTIData

In [266]:
'''Takes a PTIData instance and the excorr and emcorr LUTs as numpy arrays.
   Removes the diode, excitation LUT, and emission LUT corrections from the corrected data.
   Modifies the instances 'cor_data' member variable array.'''


def load_excorr_file(PTIData_instance):
    excorr = np.genfromtxt('PTI/correction_data/excorr.txt',
                           skip_header = 6,
                           skip_footer = 1,
                           usecols = 1)
    
    step = PTIData_instance.step_size
    excorr_range = np.arange(250, 750+1)
    xvals = np.arange(250, 750 + step , step)
    excorr = np.interp(xvals, excorr_range, excorr)
    
    min_data_wavelength = PTIData_instance.wavelengths[0]
    max_data_wavelength = PTIData_instance.wavelengths[-1]
    
    needed_wavelengths = np.where((xvals >= min_data_wavelength) & 
                                  (xvals <= max_data_wavelength))
    
    return excorr[needed_wavelengths]

def load_emcorr_file(PTIData_instance):
    emcorr = np.genfromtxt('PTI/correction_data/emcorr-sphere-quanta.txt',
                           skip_header = 6,
                           skip_footer = 1,
                           usecols = 1)
    
    step = PTIData_instance.step_size
    emcorr_range = np.arange(300, 848+2,2)
    xvals = np.arange(300, 848 + step , step )
    
    emcorr = np.interp(xvals, emcorr_range, emcorr)

    min_data_wavelength = PTIData_instance.wavelengths[0]
    max_data_wavelength = PTIData_instance.wavelengths[-1]
    
    needed_wavelengths = np.where((xvals >= min_data_wavelength) & 
                                  (xvals <= max_data_wavelength))
    
    return emcorr[needed_wavelengths]

def undo_felix_corrections(PTIData_instance, emcorr = False):
    data = copy.deepcopy(PTIData_instance)
    
    # Undo the diode correction made from the ExCorr RCQC signal
    data.cor_data  *= PTIData_instance.excorr
    
    # Undo the LUT corrections
    data.cor_data  *= load_excorr_file(PTIData_instance)
    
    if emcorr:
        data.cor_data  /= load_emcorr_file(PTIData_instance)
    
    return data

def get_felix_corrections(PTIData_instance, emcorr = False):
    data = np.ones(PTIData_instance.cor_data.size)
    
    # Undo the diode correction made from the ExCorr RCQC signal
    data  *= PTIData_instance.excorr
    
    # Undo the LUT corrections
    data *= load_excorr_file(PTIData_instance)
    
    if emcorr:
        data /= load_emcorr_file(PTIData_instance)
    
    return data
    

In [270]:
paths = ["Henry/Sphere/PPO_ETOH/EmissionScan_ETOH_ex310_2sec_160830.txt",
         "Henry/Sphere/PPO_ETOH/EmissionScan_ETOH_ex320_2sec_160830.txt",
         "Henry/Sphere/PPO_ETOH/EmissionScan_ETOH_ex330_2sec_160830.txt",
         "Henry/Sphere/PPO_ETOH/EmissionScan_ETOH_ex340_2sec_160830.txt",
         
         "Henry/Sphere/PPO_ETOH/EmissionScan_0x31gperL_PPOinETOH_ex310_2sec_160831.txt",
         "Henry/Sphere/PPO_ETOH/EmissionScan_0x31gperL_PPOinETOH_ex320_2sec_160831.txt",
         "Henry/Sphere/PPO_ETOH/EmissionScan_0x31gperL_PPOinETOH_ex330_2sec_160831.txt",
         "Henry/Sphere/PPO_ETOH/EmissionScan_0x31gperL_PPOinETOH_ex340_2sec_160831.txt",
         
         "Henry/Sphere/PPO_ETOH/EmissionScan_3x14gperL_PPOinETOH_ex310_2sec_160831.txt",
         "Henry/Sphere/PPO_ETOH/EmissionScan_3x14gperL_PPOinETOH_ex320_2sec_160831.txt",
         "Henry/Sphere/PPO_ETOH/EmissionScan_3x14gperL_PPOinETOH_ex330_2sec_160831.txt",
         "Henry/Sphere/PPO_ETOH/EmissionScan_3x14gperL_PPOinETOH_ex340_2sec_160831.txt",
         
         "Henry/Sphere/bisMSB_LAB/EmissionScan_LAB_ex350_2sec_160823.txt",
         "Henry/Sphere/bisMSB_LAB/EmissionScan_LAB_ex360_2sec_160823.txt",
         "Henry/Sphere/bisMSB_LAB/EmissionScan_LAB_ex370_2sec_160823.txt",
         "Henry/Sphere/bisMSB_LAB/EmissionScan_LAB_ex380_2sec_160823.txt",
        
         "Henry/Sphere/bisMSB_LAB/EmissionScan_bisMSBinLAB_4.47mgL_ex350_2sec_160824.txt",
         "Henry/Sphere/bisMSB_LAB/EmissionScan_bisMSBinLAB_4.47mgL_ex360_2sec_160824.txt",
         "Henry/Sphere/bisMSB_LAB/EmissionScan_bisMSBinLAB_4.47mgL_ex370_2sec_160824.txt",
         "Henry/Sphere/bisMSB_LAB/EmissionScan_bisMSBinLAB_4.47mgL_ex380_2sec_160824.txt",
         "Henry/Sphere/bisMSB_LAB/EmissionScan_bisMSBinLAB_4.47mgL_ex390_2sec_160824.txt",
         "Henry/Sphere/bisMSB_LAB/EmissionScan_bisMSBinLAB_4.47mgL_ex400_2sec_160824.txt",
         "Henry/Sphere/bisMSB_LAB/EmissionScan_bisMSBinLAB_4.47mgL_ex410_2sec_160824.txt"
         
        ]

emcorrs =[False, False, False, False,
          False, False, False, False,
          False, False, False, False,
          False, False, False, False,
          False, False, False, False, False, False, False]

ex_ranges = [[300,320],[310,330],[320,340],[330,350],
             [300,320],[310,330],[320,340],[330,350],
             [300,320],[310,330],[320,340],[330,350],
             [340,360],[350,370],[360,380],[370,390],
             [340,360],[350,370],[360,380],[370,390],[380,400],[390,410],[400,420],
            ]

em_ranges = [[320,650],[330,650],[340,650],[350,650],
             [320,650],[330,650],[340,650],[350,650],
             [320,650],[330,650],[340,650],[350,650],
             [360,650],[370,650],[380,650],[390,650],
             [360,650],[370,650],[380,650],[390,650],[400,650],[410,650],[420,650],
            ]

for i in range(len(paths)):
    test = PTIData(paths[i])
    decorrected = undo_felix_corrections(test, emcorrs[i])
    diff = decorrected.cor_data - test.raw_data
    ratio = decorrected.cor_data / test.raw_data
    
    ex_where = np.where((test.wavelengths >= ex_ranges[i][0]) &
                        (test.wavelengths <= ex_ranges[i][1]))
    
    em_where = np.where((test.wavelengths >= em_ranges[i][0]) &
                        (test.wavelengths <= em_ranges[i][1]))
        
    
    fig = plt.figure(figsize = (16,25))
    excorr_ax = plt.subplot2grid((6,2),(0,0), colspan=2)
    corr_ax = plt.subplot2grid((6,2),(1,0), colspan=2)
    ax =  plt.subplot2grid((6,2),(2,0), colspan=2)
    ax1 = plt.subplot2grid((6,2),(3,0))
    ax2 = plt.subplot2grid((6,2),(3,1))
    ax3 = plt.subplot2grid((6,2),(4,0), colspan=2)
    ax4 = plt.subplot2grid((6,2),(5,0), colspan=2)
    
    fig.suptitle("Spectra of Raw Data vs Un-corrected Data\n"+ paths[i],y=1.05)
                               
    excorr_ax.plot(test.wavelengths, test.excorr,"#433e90")
    excorr_ax.set_title("Diode Correction")
    
    corr_ax.plot(test.wavelengths, get_felix_corrections(test, emcorrs[i]), "#433e90")
    corr_ax.set_title("Total Correction")                        
    
    ax.plot(test.wavelengths, decorrected.cor_data,"#52c6c3")
    ax.plot(test.wavelengths, test.raw_data,"#696969")
    ax.legend(["De-corrected", "Raw"])
    
    ax1.plot(test.wavelengths[ex_where], decorrected.cor_data[ex_where],"#52c6c3")
    ax1.plot(test.wavelengths[ex_where], test.raw_data[ex_where],"#696969")
    ax1.set_title("Absorption Range")
    ax1.legend(["De-corrected", "Raw"])
    
    ax2.plot(test.wavelengths[em_where], decorrected.cor_data[em_where],"#52c6c3")
    ax2.plot(test.wavelengths[em_where], test.raw_data[em_where],"#696969")
    ax2.set_title("Emission Range")
    ax2.legend(["De-corrected", "Raw"])
    
    ax3.plot(test.wavelengths, diff,"k")
    ax3.set_title("Difference: Decorrected - Raw")
                
    ax4.plot(test.wavelengths, ratio,"#433e90")
    ax4.set_title("Ratio: Decorrected / Raw")
    ax4.set_ylim([0,3])
    
    for axis in [ax, ax1, ax2, ax3,ax4,excorr_ax,corr_ax]:
        axis.grid()
#         axis.set_xlim([test.wavelengths[0], test.wavelengths[-1]])
    
    plt.tight_layout()
#     plt.subplots_adjust(top=0.85)
    fname = paths[i][:-4].split("/")[-1]
    plt.savefig("/home/nmcneal/Desktop/CorrectionStudy/" + fname + ".png")
    del fig, ax, ax1, ax2,ax3, ax4,excorr_ax,corr_ax
